# CampaignWe — Data Exploration

Explore click event data from the prompt library page.  
Data source: `../data/campaignwe.db` (DuckDB, populated by `process_campaignwe.py`).

## 1 — Setup & Data Overview

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from pathlib import Path

# ── Corporate color palette ─────────────────────────────────────────
CORP_RED       = '#E60000'
CORP_RED_DARK  = '#8A000A'
BORDEAUX_I     = '#BD000C'
BLACK          = '#000000'
WHITE          = '#FFFFFF'

GRAY_I   = '#CCCABC'
GRAY_II  = '#B8B3A2'
GRAY_III = '#8E8D83'
GRAY_IV  = '#7A7870'
GRAY_V   = '#5A5D5C'
GRAY_VI  = '#404040'

PASTEL_I   = '#ECEBE4'
PASTEL_II  = '#F5F0E1'

RAG_RED   = '#BD000C'
RAG_AMBER = '#E4A911'
RAG_GREEN = '#6F7A1A'

# 20-color chart palette for multi-series charts
CHART_COLORS = [
    '#AF8626',  # Bronze50
    '#00759E',  # Lagoon60
    '#879420',  # Kiwi60
    '#4B2D58',  # Aubergine90
    '#9F8865',  # Sand50
    '#2E476B',  # Plum90
    '#469A6C',  # Sage50
    '#AD3E4A',  # Blush60
    '#8489BD',  # Lavender50
    '#0C7EC6',  # Lake50
    '#654D16',  # Bronze80
    '#804C95',  # Aubergine60
    '#45999C',  # Mint50
    '#4972AC',  # Plum60
    '#CC707A',  # Blush40
    '#295B40',  # Sage80
    '#545A9C',  # Lavender70
    '#785E4A',  # Chocolate60
    '#07476F',  # Lake90
    '#620004',  # Bordeaux90
]

# ── Matplotlib defaults ─────────────────────────────────────────────
plt.rcParams.update({
    'figure.facecolor': WHITE,
    'axes.facecolor': WHITE,
    'axes.edgecolor': GRAY_III,
    'axes.labelcolor': GRAY_VI,
    'xtick.color': GRAY_IV,
    'ytick.color': GRAY_IV,
    'text.color': GRAY_VI,
    'grid.color': PASTEL_I,
    'grid.linewidth': 0.8,
    'axes.grid': True,
    'axes.grid.axis': 'y',
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'figure.titlesize': 16,
    'figure.titleweight': 'bold',
})

# ── DuckDB connection ──────────────────────────────────────────────
DB_PATH = Path(__file__).parent.parent / 'data' / 'campaignwe.db' if '__file__' in dir() else Path('../data/campaignwe.db')
assert DB_PATH.exists(), f'Database not found at {DB_PATH}. Run process_campaignwe.py first.'
con = duckdb.connect(str(DB_PATH), read_only=True)

# Helper: run SQL and return DataFrame
def q(sql):
    return con.execute(sql).df()

print('Connected to', DB_PATH)
tables = q('SHOW TABLES')['name'].tolist()
for t in sorted(tables):
    n = q(f'SELECT COUNT(*) as n FROM {t}')['n'][0]
    print(f'  {t:<25s} {n:>10,} rows')

In [ ]:
# ── Data overview ──────────────────────────────────────────────────
overview = q("""
    SELECT
        COUNT(*)                        AS total_events,
        MIN(session_date)               AS first_date,
        MAX(session_date)               AS last_date,
        COUNT(DISTINCT session_date)    AS active_days,
        COUNT(DISTINCT gpn)             AS unique_gpns,
        COUNT(DISTINCT user_id)         AS unique_users,
        COUNT(DISTINCT session_key)     AS unique_sessions,
        COUNT(DISTINCT story_id)        AS unique_stories
    FROM events
""")
overview.T.rename(columns={0: 'Value'})

---
## 2 — Data Quality & GPN Investigation

In [ ]:
# ── GPN mapping: matched vs unmatched ──────────────────────────────
gpn_status = q("""
    SELECT
        COUNT(*)                                              AS total,
        COUNT(gpn)                                            AS has_gpn,
        SUM(CASE WHEN gpn IS NOT NULL AND hr_division IS NOT NULL THEN 1 ELSE 0 END) AS matched,
        SUM(CASE WHEN gpn IS NOT NULL AND hr_division IS NULL THEN 1 ELSE 0 END)     AS unmatched,
        SUM(CASE WHEN gpn IS NULL THEN 1 ELSE 0 END)         AS no_gpn
    FROM events
""")

labels = ['HR matched', 'GPN but no HR', 'No GPN']
values = [int(gpn_status['matched'][0]), int(gpn_status['unmatched'][0]), int(gpn_status['no_gpn'][0])]
colors = [RAG_GREEN, RAG_AMBER, GRAY_II]

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.barh(labels, values, color=colors, edgecolor=WHITE, height=0.5)
for bar, val in zip(bars, values):
    pct = 100 * val / sum(values) if sum(values) else 0
    ax.text(bar.get_width() + max(values)*0.01, bar.get_y() + bar.get_height()/2,
            f'{val:,}  ({pct:.1f}%)', va='center', fontsize=11, color=GRAY_VI)
ax.set_title('GPN → HR Mapping Coverage')
ax.set_xlim(0, max(values) * 1.3)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# ── All unmatched GPNs with event counts ───────────────────────────
unmatched_gpns = q("""
    SELECT
        gpn,
        COUNT(*) AS events,
        COUNT(DISTINCT session_key) AS sessions,
        MIN(session_date) AS first_seen,
        MAX(session_date) AS last_seen
    FROM events
    WHERE gpn IS NOT NULL AND hr_division IS NULL
    GROUP BY gpn
    ORDER BY events DESC
""")
print(f'{len(unmatched_gpns)} unique GPNs with no HR match ({unmatched_gpns["events"].sum():,} events total)')
unmatched_gpns

In [ ]:
# ── Sample events for top unmatched GPNs ───────────────────────────
# Shows email and activity to help identify who these are
sample_unmatched = q("""
    SELECT gpn, email, action_type, CP_Link_label, session_date
    FROM events
    WHERE gpn IS NOT NULL AND hr_division IS NULL
    QUALIFY ROW_NUMBER() OVER (PARTITION BY gpn ORDER BY session_date DESC) <= 3
    ORDER BY gpn, session_date DESC
""")
sample_unmatched

In [ ]:
# ── GPN format check ──────────────────────────────────────────────
gpn_formats = q("""
    SELECT
        LENGTH(gpn) AS gpn_length,
        COUNT(*) AS events,
        COUNT(DISTINCT gpn) AS unique_gpns,
        LIST(DISTINCT gpn ORDER BY gpn LIMIT 5) AS sample_gpns
    FROM events
    WHERE gpn IS NOT NULL
    GROUP BY 1
    ORDER BY 1
""")
print('GPN length distribution:')
gpn_formats

In [ ]:
# ── Field NULL rates ───────────────────────────────────────────────
fields = ['gpn', 'email', 'session_id', 'user_id', 'story_id', 'action_type',
          'hr_division', 'hr_region', 'hr_country', 'hr_job_family']
# Only check fields that exist
all_cols = q('DESCRIBE events')['column_name'].tolist()
fields = [f for f in fields if f in all_cols]

null_rates = q(f"""
    SELECT
        {', '.join(f"SUM(CASE WHEN {f} IS NOT NULL THEN 1 ELSE 0 END) AS \"{f}\"" for f in fields)}
    FROM events
""")
total = q('SELECT COUNT(*) AS n FROM events')['n'][0]
null_df = pd.DataFrame({
    'field': fields,
    'non_null': [int(null_rates[f][0]) for f in fields],
    'null': [int(total - null_rates[f][0]) for f in fields],
    'fill_rate': [f"{100*int(null_rates[f][0])/total:.1f}%" for f in fields]
})
null_df

In [ ]:
# ── Events per day (timeline) ─────────────────────────────────────
daily = q("""
    SELECT session_date AS date, COUNT(*) AS events, COUNT(DISTINCT gpn) AS users
    FROM events
    GROUP BY 1 ORDER BY 1
""")
daily['date'] = pd.to_datetime(daily['date'])

fig, ax1 = plt.subplots(figsize=(12, 5))
ax1.fill_between(daily['date'], daily['events'], alpha=0.3, color=CORP_RED)
ax1.plot(daily['date'], daily['events'], color=CORP_RED, linewidth=1.5, label='Events')
ax1.set_ylabel('Events', color=CORP_RED)
ax1.tick_params(axis='y', labelcolor=CORP_RED)

ax2 = ax1.twinx()
ax2.plot(daily['date'], daily['users'], color=CHART_COLORS[1], linewidth=1.5, linestyle='--', label='Unique users')
ax2.set_ylabel('Unique users (GPN)', color=CHART_COLORS[1])
ax2.tick_params(axis='y', labelcolor=CHART_COLORS[1])

ax1.set_title('Daily Activity')
fig.legend(loc='upper right', bbox_to_anchor=(0.95, 0.95))
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

---
## 3 — Click Distribution & Behavior

In [ ]:
# ── Action type distribution ───────────────────────────────────────
actions = q("""
    SELECT COALESCE(action_type, '(null)') AS action_type, COUNT(*) AS cnt
    FROM events
    GROUP BY 1 ORDER BY cnt DESC
""")

fig, ax = plt.subplots(figsize=(10, 5))
colors_act = [CHART_COLORS[i % len(CHART_COLORS)] for i in range(len(actions))]
bars = ax.bar(actions['action_type'], actions['cnt'], color=colors_act, edgecolor=WHITE)
for bar, val in zip(bars, actions['cnt']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(actions['cnt'])*0.01,
            f'{val:,}', ha='center', va='bottom', fontsize=10, color=GRAY_VI)
ax.set_title('Action Type Distribution')
ax.set_ylabel('Events')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# ── Link type distribution ─────────────────────────────────────────
link_types = q("""
    SELECT COALESCE(CP_Link_Type, '(blank)') AS link_type, COUNT(*) AS cnt
    FROM events
    GROUP BY 1 ORDER BY cnt DESC
""")

fig, ax = plt.subplots(figsize=(10, 4))
ax.barh(link_types['link_type'], link_types['cnt'], color=CHART_COLORS[1], edgecolor=WHITE, height=0.5)
for i, (val, lt) in enumerate(zip(link_types['cnt'], link_types['link_type'])):
    ax.text(val + max(link_types['cnt'])*0.01, i, f'{val:,}', va='center', fontsize=10, color=GRAY_VI)
ax.set_title('Link Type Distribution (CP_Link_Type)')
ax.set_xlabel('Events')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# ── Events by hour of day (CET) ───────────────────────────────────
hourly = q("""
    SELECT event_hour, COUNT(*) AS cnt
    FROM events
    GROUP BY 1 ORDER BY 1
""")

fig, ax = plt.subplots(figsize=(12, 4))
bars = ax.bar(hourly['event_hour'], hourly['cnt'], color=CORP_RED, edgecolor=WHITE, width=0.8)
ax.set_title('Events by Hour of Day (CET)')
ax.set_xlabel('Hour')
ax.set_ylabel('Events')
ax.set_xticks(range(0, 24))
plt.tight_layout()
plt.show()

In [ ]:
# ── Events by weekday ──────────────────────────────────────────────
weekday = q("""
    SELECT event_weekday, event_weekday_num, COUNT(*) AS cnt
    FROM events
    GROUP BY 1, 2 ORDER BY 2
""")

fig, ax = plt.subplots(figsize=(10, 4))
colors_wd = [CORP_RED if d <= 5 else GRAY_III for d in weekday['event_weekday_num']]
ax.bar(weekday['event_weekday'], weekday['cnt'], color=colors_wd, edgecolor=WHITE)
ax.set_title('Events by Weekday (CET)')
ax.set_ylabel('Events')
plt.tight_layout()
plt.show()

In [ ]:
# ── Heatmap: weekday × hour ────────────────────────────────────────
heatmap_data = q("""
    SELECT event_weekday_num, event_weekday, event_hour, COUNT(*) AS cnt
    FROM events
    GROUP BY 1, 2, 3
""")

pivot = heatmap_data.pivot_table(index=['event_weekday_num', 'event_weekday'],
                                  columns='event_hour', values='cnt', fill_value=0)
pivot = pivot.sort_index(level=0)
day_labels = [row[1] for row in pivot.index]

fig, ax = plt.subplots(figsize=(14, 5))
from matplotlib.colors import LinearSegmentedColormap
corp_cmap = LinearSegmentedColormap.from_list('corp', [WHITE, '#F5F0E1', RAG_AMBER, CORP_RED, CORP_RED_DARK])
im = ax.imshow(pivot.values, aspect='auto', cmap=corp_cmap)
ax.set_yticks(range(len(day_labels)))
ax.set_yticklabels(day_labels)
ax.set_xticks(range(24))
ax.set_xticklabels(range(24))
ax.set_xlabel('Hour (CET)')
ax.set_title('Activity Heatmap: Weekday × Hour')
plt.colorbar(im, ax=ax, label='Events', shrink=0.8)
plt.tight_layout()
plt.show()

In [ ]:
# ── Session length distribution ────────────────────────────────────
session_lengths = q("""
    SELECT session_key, COUNT(*) AS events_in_session
    FROM events
    GROUP BY 1
""")

fig, ax = plt.subplots(figsize=(10, 4))
max_bin = min(session_lengths['events_in_session'].quantile(0.95), 50)
ax.hist(session_lengths['events_in_session'], bins=range(1, int(max_bin)+2),
        color=CHART_COLORS[0], edgecolor=WHITE, alpha=0.9)
ax.set_title('Session Length Distribution (events per session)')
ax.set_xlabel('Events in session')
ax.set_ylabel('Number of sessions')
ax.axvline(session_lengths['events_in_session'].median(), color=CORP_RED, linestyle='--',
           label=f"Median: {session_lengths['events_in_session'].median():.0f}")
ax.legend()
plt.tight_layout()
plt.show()

print(f"Sessions: {len(session_lengths):,}")
print(session_lengths['events_in_session'].describe().round(1))

In [ ]:
# ── Time between clicks ────────────────────────────────────────────
time_buckets = q("""
    SELECT time_since_prev_bucket AS bucket, COUNT(*) AS cnt
    FROM events
    WHERE time_since_prev_bucket IS NOT NULL
    GROUP BY 1
""")

bucket_order = ['First Event', '< 0.5s', '0.5-1s', '1-2s', '2-5s', '5-10s', '10-30s', '30-60s', '> 60s']
time_buckets['sort'] = time_buckets['bucket'].map({b: i for i, b in enumerate(bucket_order)})
time_buckets = time_buckets.sort_values('sort')

fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(time_buckets['bucket'], time_buckets['cnt'], color=CHART_COLORS[2], edgecolor=WHITE)
ax.set_title('Time Between Consecutive Clicks')
ax.set_ylabel('Events')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# ── "Other" action labels — what are they? ─────────────────────────
other_labels = q("""
    SELECT CP_Link_label, COUNT(*) AS cnt
    FROM events
    WHERE action_type = 'Other'
    GROUP BY 1 ORDER BY cnt DESC
    LIMIT 30
""")
print(f'Top "Other" labels ({other_labels["cnt"].sum():,} events):')
other_labels

---
## 4 — Engagement by Division & Region

In [ ]:
# ── Events per division (top 15) ──────────────────────────────────
div_events = q("""
    SELECT COALESCE(hr_division, '(unknown)') AS division,
           COUNT(*) AS events,
           COUNT(DISTINCT gpn) AS users
    FROM events
    GROUP BY 1 ORDER BY events DESC
    LIMIT 15
""")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Events
colors_div = [GRAY_II if d == '(unknown)' else CORP_RED for d in div_events['division']]
ax1.barh(div_events['division'][::-1], div_events['events'][::-1],
         color=colors_div[::-1], edgecolor=WHITE, height=0.6)
for i, val in enumerate(div_events['events'][::-1]):
    ax1.text(val + max(div_events['events'])*0.01, i, f'{val:,}',
             va='center', fontsize=9, color=GRAY_VI)
ax1.set_title('Events by Division')
ax1.set_xlabel('Events')

# Unique users
colors_div2 = [GRAY_II if d == '(unknown)' else CHART_COLORS[1] for d in div_events['division']]
ax2.barh(div_events['division'][::-1], div_events['users'][::-1],
         color=colors_div2[::-1], edgecolor=WHITE, height=0.6)
for i, val in enumerate(div_events['users'][::-1]):
    ax2.text(val + max(div_events['users'])*0.01, i, f'{val:,}',
             va='center', fontsize=9, color=GRAY_VI)
ax2.set_title('Unique Users by Division')
ax2.set_xlabel('Users (distinct GPN)')

plt.tight_layout()
plt.show()

In [ ]:
# ── Engagement depth per division ──────────────────────────────────
div_depth = q("""
    SELECT
        COALESCE(hr_division, '(unknown)') AS division,
        COUNT(*) AS total_events,
        COUNT(DISTINCT gpn) AS unique_users,
        ROUND(COUNT(*) * 1.0 / NULLIF(COUNT(DISTINCT gpn), 0), 1) AS events_per_user,
        COUNT(DISTINCT story_id) AS stories_touched,
        SUM(CASE WHEN action_type = 'Read' THEN 1 ELSE 0 END) AS reads,
        SUM(CASE WHEN action_type = 'Like' THEN 1 ELSE 0 END) AS likes,
        SUM(CASE WHEN action_type = 'Share' THEN 1 ELSE 0 END) AS shares
    FROM events
    GROUP BY 1
    ORDER BY total_events DESC
""")
div_depth

In [ ]:
# ── Events per region ──────────────────────────────────────────────
reg_events = q("""
    SELECT COALESCE(hr_region, '(unknown)') AS region,
           COUNT(*) AS events,
           COUNT(DISTINCT gpn) AS users
    FROM events
    GROUP BY 1 ORDER BY events DESC
""")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, max(4, len(reg_events)*0.4)))

colors_reg = [GRAY_II if r == '(unknown)' else CHART_COLORS[0] for r in reg_events['region']]
ax1.barh(reg_events['region'][::-1], reg_events['events'][::-1],
         color=colors_reg[::-1], edgecolor=WHITE, height=0.6)
for i, val in enumerate(reg_events['events'][::-1]):
    ax1.text(val + max(reg_events['events'])*0.01, i, f'{val:,}',
             va='center', fontsize=9, color=GRAY_VI)
ax1.set_title('Events by Region')
ax1.set_xlabel('Events')

colors_reg2 = [GRAY_II if r == '(unknown)' else CHART_COLORS[6] for r in reg_events['region']]
ax2.barh(reg_events['region'][::-1], reg_events['users'][::-1],
         color=colors_reg2[::-1], edgecolor=WHITE, height=0.6)
for i, val in enumerate(reg_events['users'][::-1]):
    ax2.text(val + max(reg_events['users'])*0.01, i, f'{val:,}',
             va='center', fontsize=9, color=GRAY_VI)
ax2.set_title('Unique Users by Region')
ax2.set_xlabel('Users (distinct GPN)')

plt.tight_layout()
plt.show()

---
## 5 — Story Engagement

In [ ]:
# ── Story ranking by reads ─────────────────────────────────────────
story_reads = q("""
    SELECT story_id,
           SUM(CASE WHEN action_type = 'Read' THEN 1 ELSE 0 END) AS reads,
           COUNT(DISTINCT gpn) AS unique_readers,
           COUNT(*) AS total_interactions
    FROM events
    WHERE story_id IS NOT NULL
    GROUP BY 1
    ORDER BY reads DESC
""")

top_n = min(20, len(story_reads))
top_stories = story_reads.head(top_n)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, max(5, top_n*0.35)))

ax1.barh(top_stories['story_id'].astype(str)[::-1], top_stories['reads'][::-1],
         color=CORP_RED, edgecolor=WHITE, height=0.6)
ax1.set_title(f'Top {top_n} Stories by Reads')
ax1.set_xlabel('Read events')
ax1.set_ylabel('Story ID')

ax2.barh(top_stories['story_id'].astype(str)[::-1], top_stories['unique_readers'][::-1],
         color=CHART_COLORS[1], edgecolor=WHITE, height=0.6)
ax2.set_title(f'Top {top_n} Stories by Unique Readers')
ax2.set_xlabel('Unique readers (GPN)')
ax2.set_ylabel('Story ID')

plt.tight_layout()
plt.show()

print(f'\n{len(story_reads)} stories total, {story_reads["reads"].sum():,} total reads')
story_reads.head(20)

In [ ]:
# ── Engagement funnel per story (top 10) ───────────────────────────
funnel = q("""
    SELECT story_id,
           SUM(CASE WHEN action_type = 'Read' THEN 1 ELSE 0 END) AS reads,
           SUM(CASE WHEN action_type = 'Hide' THEN 1 ELSE 0 END) AS hides,
           SUM(CASE WHEN action_type = 'Like' THEN 1 ELSE 0 END) AS likes,
           SUM(CASE WHEN action_type = 'Share' THEN 1 ELSE 0 END) AS shares,
           SUM(CASE WHEN action_type = 'View Prompt' THEN 1 ELSE 0 END) AS view_prompts
    FROM events
    WHERE story_id IS NOT NULL
    GROUP BY 1
    ORDER BY reads DESC
    LIMIT 10
""")

fig, ax = plt.subplots(figsize=(14, 6))
x = np.arange(len(funnel))
width = 0.15
metrics = ['reads', 'hides', 'likes', 'shares', 'view_prompts']
labels = ['Read', 'Hide', 'Like', 'Share', 'View Prompt']
colors_funnel = [CORP_RED, GRAY_III, RAG_GREEN, CHART_COLORS[1], CHART_COLORS[0]]

for i, (metric, label, color) in enumerate(zip(metrics, labels, colors_funnel)):
    offset = (i - len(metrics)/2 + 0.5) * width
    ax.bar(x + offset, funnel[metric], width, label=label, color=color, edgecolor=WHITE)

ax.set_xticks(x)
ax.set_xticklabels([f'Story {s}' for s in funnel['story_id']], rotation=30, ha='right')
ax.set_title('Engagement Funnel — Top 10 Stories')
ax.set_ylabel('Events')
ax.legend()
plt.tight_layout()
plt.show()

---
## 6 — Division × Story & Region × Story

In [ ]:
# ── Heatmap: top stories × top divisions (by reads) ───────────────
div_story = q("""
    WITH top_stories AS (
        SELECT story_id FROM events
        WHERE story_id IS NOT NULL AND action_type = 'Read'
        GROUP BY 1 ORDER BY COUNT(*) DESC LIMIT 10
    ),
    top_divs AS (
        SELECT hr_division FROM events
        WHERE hr_division IS NOT NULL
        GROUP BY 1 ORDER BY COUNT(*) DESC LIMIT 10
    )
    SELECT hr_division AS division, story_id,
           SUM(CASE WHEN action_type = 'Read' THEN 1 ELSE 0 END) AS reads
    FROM events
    WHERE story_id IN (SELECT story_id FROM top_stories)
      AND hr_division IN (SELECT hr_division FROM top_divs)
    GROUP BY 1, 2
""")

if len(div_story) > 0:
    pivot_ds = div_story.pivot_table(index='division', columns='story_id', values='reads', fill_value=0)
    # Sort divisions by total reads
    pivot_ds = pivot_ds.loc[pivot_ds.sum(axis=1).sort_values(ascending=False).index]

    fig, ax = plt.subplots(figsize=(14, 8))
    im = ax.imshow(pivot_ds.values, aspect='auto', cmap=corp_cmap)
    ax.set_xticks(range(pivot_ds.shape[1]))
    ax.set_xticklabels([f'Story {c}' for c in pivot_ds.columns], rotation=45, ha='right')
    ax.set_yticks(range(pivot_ds.shape[0]))
    ax.set_yticklabels(pivot_ds.index)
    # Annotate cells
    for i in range(pivot_ds.shape[0]):
        for j in range(pivot_ds.shape[1]):
            val = pivot_ds.values[i, j]
            if val > 0:
                text_color = WHITE if val > pivot_ds.values.max() * 0.6 else GRAY_VI
                ax.text(j, i, f'{val:.0f}', ha='center', va='center', fontsize=9, color=text_color)
    ax.set_title('Reads: Division × Story (Top 10 each)')
    plt.colorbar(im, ax=ax, label='Reads', shrink=0.8)
    plt.tight_layout()
    plt.show()
else:
    print('No division × story data available.')

In [ ]:
# ── Heatmap: top stories × regions (by reads) ─────────────────────
reg_story = q("""
    WITH top_stories AS (
        SELECT story_id FROM events
        WHERE story_id IS NOT NULL AND action_type = 'Read'
        GROUP BY 1 ORDER BY COUNT(*) DESC LIMIT 10
    )
    SELECT hr_region AS region, story_id,
           SUM(CASE WHEN action_type = 'Read' THEN 1 ELSE 0 END) AS reads
    FROM events
    WHERE story_id IN (SELECT story_id FROM top_stories)
      AND hr_region IS NOT NULL
    GROUP BY 1, 2
""")

if len(reg_story) > 0:
    pivot_rs = reg_story.pivot_table(index='region', columns='story_id', values='reads', fill_value=0)
    pivot_rs = pivot_rs.loc[pivot_rs.sum(axis=1).sort_values(ascending=False).index]

    fig, ax = plt.subplots(figsize=(14, max(4, len(pivot_rs)*0.5)))
    im = ax.imshow(pivot_rs.values, aspect='auto', cmap=corp_cmap)
    ax.set_xticks(range(pivot_rs.shape[1]))
    ax.set_xticklabels([f'Story {c}' for c in pivot_rs.columns], rotation=45, ha='right')
    ax.set_yticks(range(pivot_rs.shape[0]))
    ax.set_yticklabels(pivot_rs.index)
    for i in range(pivot_rs.shape[0]):
        for j in range(pivot_rs.shape[1]):
            val = pivot_rs.values[i, j]
            if val > 0:
                text_color = WHITE if val > pivot_rs.values.max() * 0.6 else GRAY_VI
                ax.text(j, i, f'{val:.0f}', ha='center', va='center', fontsize=9, color=text_color)
    ax.set_title('Reads: Region × Story (Top 10 Stories)')
    plt.colorbar(im, ax=ax, label='Reads', shrink=0.8)
    plt.tight_layout()
    plt.show()
else:
    print('No region × story data available.')

---
## 7 — Engagement Over Time

In [ ]:
# ── Daily unique users by top 5 divisions ─────────────────────────
top5_divs = q("""
    SELECT hr_division FROM events
    WHERE hr_division IS NOT NULL
    GROUP BY 1 ORDER BY COUNT(DISTINCT gpn) DESC LIMIT 5
""")['hr_division'].tolist()

div_daily = q("""
    SELECT session_date AS date, hr_division AS division, COUNT(DISTINCT gpn) AS users
    FROM events
    WHERE hr_division IS NOT NULL
    GROUP BY 1, 2
    ORDER BY 1
""")
div_daily['date'] = pd.to_datetime(div_daily['date'])

fig, ax = plt.subplots(figsize=(14, 6))
for i, div in enumerate(top5_divs):
    subset = div_daily[div_daily['division'] == div]
    ax.plot(subset['date'], subset['users'], label=div, color=CHART_COLORS[i], linewidth=1.5)
ax.set_title('Daily Unique Users — Top 5 Divisions')
ax.set_ylabel('Unique users')
ax.legend(fontsize=9, loc='upper left', bbox_to_anchor=(1, 1))
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

In [ ]:
# ── Story reads over time for top 5 stories ──────────────────────
top5_stories = q("""
    SELECT story_id FROM events
    WHERE story_id IS NOT NULL AND action_type = 'Read'
    GROUP BY 1 ORDER BY COUNT(*) DESC LIMIT 5
""")['story_id'].tolist()

story_daily = q("""
    SELECT session_date AS date, story_id,
           SUM(CASE WHEN action_type = 'Read' THEN 1 ELSE 0 END) AS reads
    FROM events
    WHERE story_id IS NOT NULL
    GROUP BY 1, 2
    ORDER BY 1
""")
story_daily['date'] = pd.to_datetime(story_daily['date'])

fig, ax = plt.subplots(figsize=(14, 6))
for i, sid in enumerate(top5_stories):
    subset = story_daily[story_daily['story_id'] == sid]
    ax.plot(subset['date'], subset['reads'], label=f'Story {sid}', color=CHART_COLORS[i], linewidth=1.5)
ax.set_title('Daily Reads — Top 5 Stories')
ax.set_ylabel('Read events')
ax.legend(fontsize=9, loc='upper left', bbox_to_anchor=(1, 1))
fig.autofmt_xdate()
plt.tight_layout()
plt.show()

In [ ]:
# ── Key metrics summary table per division ─────────────────────────
summary = q("""
    SELECT
        COALESCE(hr_division, '(unknown)') AS division,
        COUNT(*) AS total_events,
        COUNT(DISTINCT gpn) AS unique_users,
        COUNT(DISTINCT story_id) AS stories_read,
        ROUND(SUM(CASE WHEN action_type = 'Read' THEN 1.0 ELSE 0 END) / NULLIF(COUNT(DISTINCT gpn), 0), 1) AS reads_per_user,
        SUM(CASE WHEN action_type = 'Read' THEN 1 ELSE 0 END) AS total_reads,
        SUM(CASE WHEN action_type = 'Like' THEN 1 ELSE 0 END) AS total_likes,
        SUM(CASE WHEN action_type = 'Share' THEN 1 ELSE 0 END) AS total_shares,
        (
            SELECT story_id FROM events e2
            WHERE e2.hr_division = events.hr_division
              AND e2.story_id IS NOT NULL AND e2.action_type = 'Read'
            GROUP BY story_id ORDER BY COUNT(*) DESC LIMIT 1
        ) AS top_story
    FROM events
    GROUP BY hr_division
    ORDER BY total_events DESC
""")
print('Division Engagement Summary:')
summary

In [ ]:
# ── Clean up ───────────────────────────────────────────────────────
con.close()
print('Done. Connection closed.')